# Module 6: GitHub Actions & CI/CD Introduction

**Estimated Time**: 4-5 hours

**Difficulty**: Intermediate to Advanced

---

## Learning Objectives

By the end of this module, you will be able to:

1. Understand CI/CD concepts and benefits
2. Create your first GitHub Actions workflow
3. Set up automated testing for your projects
4. Build and deploy applications automatically
5. Use actions from the GitHub Marketplace
6. Manage secrets and environment variables
7. Implement workflow best practices
8. Debug and troubleshoot workflows

---

## 1. What is CI/CD?

### Continuous Integration (CI)

**Automatically build and test code** when changes are pushed.

```
Developer workflow WITHOUT CI:
┌─────────┐     ┌─────────┐     ┌──────────┐
│  Code   │ ──► │  Push   │ ──► │  Manual  │
│ Changes │     │ to repo │     │  Testing │ ❌ Error found days later
└─────────┘     └─────────┘     └──────────┘

Developer workflow WITH CI:
┌─────────┐     ┌─────────┐     ┌──────────┐     ┌──────────┐
│  Code   │ ──► │  Push   │ ──► │   Auto   │ ──► │ Instant  │
│ Changes │     │ to repo │     │   Test   │     │ Feedback │ ✅ Catch bugs early!
└─────────┘     └─────────┘     └──────────┘     └──────────┘
```

**Benefits:**
- Catch bugs early
- Faster development
- Better code quality
- Confidence in changes

### Continuous Deployment (CD)

**Automatically deploy code** when tests pass.

```
┌──────┐     ┌──────┐     ┌────────┐     ┌──────────┐
│ Code │ ──► │ Test │ ──► │ Build  │ ──► │  Deploy  │
│      │     │  ✅  │     │   ✅   │     │ to Prod  │
└──────┘     └──────┘     └────────┘     └──────────┘
        Automated Pipeline
```

**Benefits:**
- Faster releases
- Reduced manual work
- Consistent deployments
- Quick rollbacks

---

## 2. GitHub Actions Overview

### What is GitHub Actions?

**GitHub Actions** is a CI/CD platform built into GitHub that automates workflows.

### Key Concepts

**Workflow**: Automated process defined in YAML
```yaml
name: My Workflow
on: [push]
jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - run: npm test
```

**Event**: Trigger that starts a workflow
- `push`: Code pushed to repository
- `pull_request`: PR opened/updated
- `schedule`: Run on schedule (cron)
- `workflow_dispatch`: Manual trigger

**Job**: Set of steps that run on same runner
- Can run in parallel or sequentially
- Each job runs in fresh environment

**Step**: Individual task in a job
- Run commands or use actions
- Execute sequentially

**Action**: Reusable unit of code
- From marketplace or custom
- `uses: actions/checkout@v3`

**Runner**: Server that runs workflows
- GitHub-hosted (Ubuntu, Windows, macOS)
- Self-hosted (your own servers)

### Workflow Hierarchy

```
Workflow
  ├── Event (trigger)
  └── Jobs
       ├── Job 1
       │    ├── Step 1
       │    ├── Step 2
       │    └── Step 3
       └── Job 2
            ├── Step 1
            └── Step 2
```

---

## 3. Your First Workflow

### Hello World Workflow

Create `.github/workflows/hello.yml`:

In [ ]:
%%writefile .github/workflows/hello.yml
name: Hello World

# When to run this workflow
on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

# What to do
jobs:
  greet:
    # Type of runner
    runs-on: ubuntu-latest
    
    # Steps to execute
    steps:
      # Step 1: Print a message
      - name: Say hello
        run: echo "Hello, GitHub Actions!"
      
      # Step 2: Print date
      - name: Show date
        run: date
      
      # Step 3: List files
      - name: List files
        run: ls -la

### Understanding the Syntax

**`name`**: Workflow name (shows in Actions tab)

**`on`**: Events that trigger workflow
```yaml
on: push                    # Any push
on: [push, pull_request]    # Multiple events
on:
  push:
    branches: [ main ]      # Only main branch
```

**`jobs`**: Collection of jobs to run

**`runs-on`**: Operating system
- `ubuntu-latest` (most common)
- `windows-latest`
- `macos-latest`

**`steps`**: Sequence of tasks
- `name`: Description of step
- `run`: Shell command to execute
- `uses`: Action to use

---

## 4. Testing Workflow: Python Project

### Complete Python CI Workflow

In [ ]:
%%writefile .github/workflows/python-ci.yml
name: Python CI

on:
  push:
    branches: [ main, develop ]
  pull_request:
    branches: [ main, develop ]

jobs:
  test:
    runs-on: ubuntu-latest
    
    strategy:
      matrix:
        python-version: ['3.8', '3.9', '3.10', '3.11']
    
    steps:
    # Checkout code
    - name: Checkout code
      uses: actions/checkout@v3
    
    # Set up Python
    - name: Set up Python ${{ matrix.python-version }}
      uses: actions/setup-python@v4
      with:
        python-version: ${{ matrix.python-version }}
    
    # Install dependencies
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install pytest pytest-cov
        if [ -f requirements.txt ]; then pip install -r requirements.txt; fi
    
    # Run linting
    - name: Lint with pylint
      run: |
        pip install pylint
        pylint src/ --exit-zero
    
    # Run tests
    - name: Test with pytest
      run: |
        pytest tests/ --cov=src/ --cov-report=xml
    
    # Upload coverage
    - name: Upload coverage to Codecov
      uses: codecov/codecov-action@v3
      with:
        file: ./coverage.xml
        fail_ci_if_error: true

### Breaking Down the Workflow

**Matrix Strategy:**
```yaml
strategy:
  matrix:
    python-version: ['3.8', '3.9', '3.10', '3.11']
```
- Runs job multiple times with different Python versions
- 4 parallel jobs in this case
- Great for cross-version compatibility

**Common Actions:**

1. **`actions/checkout@v3`**
   - Checks out your repository
   - Always needed to access your code

2. **`actions/setup-python@v4`**
   - Installs specified Python version
   - Caches pip dependencies

3. **Multi-line commands:**
   ```yaml
   run: |
     command1
     command2
     command3
   ```

---

## 5. Common Workflow Triggers

### Push Events

In [ ]:
# Any push to any branch
on: push

# Push to specific branches
on:
  push:
    branches:
      - main
      - develop
      - 'release/**'  # release/1.0, release/2.0, etc.

# Push to specific paths
on:
  push:
    paths:
      - 'src/**'
      - 'tests/**'
    paths-ignore:
      - 'docs/**'
      - '**.md'

### Pull Request Events

In [ ]:
# All PR events
on: pull_request

# Specific PR events
on:
  pull_request:
    types:
      - opened
      - synchronize  # New commits pushed
      - reopened
    branches:
      - main

### Schedule (Cron)

In [ ]:
%%writefile .github/workflows/scheduled.yml
name: Nightly Build

on:
  schedule:
    # Run at 2 AM UTC every day
    - cron: '0 2 * * *'
    # Run at 9 AM UTC on Mondays
    - cron: '0 9 * * 1'

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - run: make build

**Cron syntax:**
```
┌───────── minute (0-59)
│ ┌─────── hour (0-23)
│ │ ┌───── day of month (1-31)
│ │ │ ┌─── month (1-12)
│ │ │ │ ┌─ day of week (0-6, Sunday=0)
│ │ │ │ │
* * * * *

Examples:
0 2 * * *      # 2 AM daily
0 0 * * 0      # Midnight every Sunday
*/15 * * * *   # Every 15 minutes
0 9 1 * *      # 9 AM on first of month
```

### Manual Trigger

In [ ]:
%%writefile .github/workflows/manual.yml
name: Manual Deployment

on:
  workflow_dispatch:
    inputs:
      environment:
        description: 'Environment to deploy to'
        required: true
        type: choice
        options:
          - development
          - staging
          - production
      version:
        description: 'Version to deploy'
        required: true
        default: 'latest'

jobs:
  deploy:
    runs-on: ubuntu-latest
    steps:
      - name: Deploy to ${{ inputs.environment }}
        run: |
          echo "Deploying version ${{ inputs.version }}"
          echo "to ${{ inputs.environment }}"

---

## 6. Working with Secrets

### Why Secrets?

Never hardcode sensitive data:
- ❌ API keys
- ❌ Passwords  
- ❌ Tokens
- ❌ Certificates

### Adding Secrets

**Repository Settings:**
```
Repository → Settings → Secrets and variables → Actions → New repository secret

Name: API_KEY
Value: your-secret-key-here
```

### Using Secrets in Workflows

In [ ]:
%%writefile .github/workflows/deploy.yml
name: Deploy

on:
  push:
    branches: [ main ]

jobs:
  deploy:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      
      # Use secret in environment variable
      - name: Deploy to server
        env:
          API_KEY: ${{ secrets.API_KEY }}
          DATABASE_URL: ${{ secrets.DATABASE_URL }}
        run: |
          echo "Deploying with credentials..."
          # API_KEY is available here
          ./deploy.sh
      
      # Use secret in action
      - name: Publish to NPM
        uses: JS-DevTools/npm-publish@v1
        with:
          token: ${{ secrets.NPM_TOKEN }}

### Environment Secrets

Different secrets for different environments:

In [ ]:
%%writefile .github/workflows/multi-env.yml
name: Multi-Environment Deploy

on:
  push:
    branches: [ main ]

jobs:
  deploy-staging:
    runs-on: ubuntu-latest
    environment: staging
    steps:
      - uses: actions/checkout@v3
      - name: Deploy to staging
        env:
          API_KEY: ${{ secrets.API_KEY }}  # staging API_KEY
        run: ./deploy.sh
  
  deploy-production:
    runs-on: ubuntu-latest
    environment: production
    needs: deploy-staging  # Wait for staging
    steps:
      - uses: actions/checkout@v3
      - name: Deploy to production
        env:
          API_KEY: ${{ secrets.API_KEY }}  # production API_KEY
        run: ./deploy.sh

**Built-in secrets:**
- `${{ secrets.GITHUB_TOKEN }}`: Automatically provided
- Authenticate with GitHub API
- Create releases, comments, etc.

---

## 7. Caching Dependencies

Speed up workflows by caching dependencies.

### Python Dependencies

In [ ]:
%%writefile .github/workflows/cache-python.yml
name: Python with Caching

on: [push]

jobs:
  test:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      
      - name: Set up Python
        uses: actions/setup-python@v4
        with:
          python-version: '3.10'
          cache: 'pip'  # Automatically cache pip dependencies
      
      - name: Install dependencies
        run: pip install -r requirements.txt
      
      - name: Run tests
        run: pytest

### Manual Caching

In [ ]:
%%writefile .github/workflows/manual-cache.yml
name: Manual Caching

on: [push]

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      
      # Cache specific directories
      - name: Cache node modules
        uses: actions/cache@v3
        with:
          path: |
            ~/.npm
            node_modules
          key: ${{ runner.os }}-node-${{ hashFiles('**/package-lock.json') }}
          restore-keys: |
            ${{ runner.os }}-node-
      
      - name: Install dependencies
        run: npm install
      
      - name: Build
        run: npm run build

**Cache benefits:**
- ⚡ Faster builds (seconds instead of minutes)
- 💰 Lower costs (less runner time)
- 🔄 Automatic invalidation when dependencies change

---

## 8. Artifacts

Save files created during workflow for later use.

### Upload Artifacts

In [ ]:
%%writefile .github/workflows/artifacts.yml
name: Build and Save Artifacts

on: [push]

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      
      - name: Build application
        run: |
          mkdir -p dist
          echo "Built app" > dist/app.txt
          echo "Log file" > build.log
      
      # Upload build artifacts
      - name: Upload build artifacts
        uses: actions/upload-artifact@v3
        with:
          name: build-artifacts
          path: |
            dist/
            build.log
          retention-days: 30
      
      # Upload test coverage
      - name: Upload coverage
        uses: actions/upload-artifact@v3
        with:
          name: coverage-report
          path: coverage/
          if-no-files-found: warn

### Download Artifacts in Another Job

In [ ]:
%%writefile .github/workflows/artifact-download.yml
name: Build and Deploy

on: [push]

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - name: Build
        run: npm run build
      
      - name: Upload build
        uses: actions/upload-artifact@v3
        with:
          name: dist
          path: dist/
  
  deploy:
    runs-on: ubuntu-latest
    needs: build  # Wait for build job
    steps:
      # Download artifact from build job
      - name: Download build
        uses: actions/download-artifact@v3
        with:
          name: dist
          path: dist/
      
      - name: Deploy
        run: |
          ls -la dist/
          # Deploy the built files

**Artifact use cases:**
- Build outputs
- Test reports
- Coverage reports
- Screenshots
- Logs

---

## 9. Job Dependencies

### Sequential Jobs

In [ ]:
%%writefile .github/workflows/sequential.yml
name: Sequential Pipeline

on: [push]

jobs:
  test:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - run: npm test
  
  build:
    runs-on: ubuntu-latest
    needs: test  # Wait for test to complete
    steps:
      - uses: actions/checkout@v3
      - run: npm run build
  
  deploy:
    runs-on: ubuntu-latest
    needs: build  # Wait for build to complete
    steps:
      - uses: actions/checkout@v3
      - run: ./deploy.sh

### Multiple Dependencies

In [ ]:
%%writefile .github/workflows/multi-deps.yml
name: Complex Pipeline

on: [push]

jobs:
  lint:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - run: npm run lint
  
  test:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - run: npm test
  
  security:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - run: npm audit
  
  deploy:
    runs-on: ubuntu-latest
    needs: [lint, test, security]  # Wait for all three
    steps:
      - uses: actions/checkout@v3
      - run: ./deploy.sh

**Execution flow:**
```
┌──────┐  ┌──────┐  ┌──────────┐
│ Lint │  │ Test │  │ Security │  (Parallel)
└───┬──┘  └───┬──┘  └────┬─────┘
    └─────────┴──────────┘
              │
         ┌────▼────┐
         │ Deploy  │  (Sequential)
         └─────────┘
```

---

## 10. Marketplace Actions

Reuse actions from the [GitHub Marketplace](https://github.com/marketplace?type=actions).

### Popular Actions

**1. Checkout Code**
```yaml
- uses: actions/checkout@v3
  with:
    fetch-depth: 0  # Full history
```

**2. Setup Languages**
```yaml
- uses: actions/setup-python@v4
  with:
    python-version: '3.10'

- uses: actions/setup-node@v3
  with:
    node-version: '18'

- uses: actions/setup-java@v3
  with:
    java-version: '17'
    distribution: 'temurin'
```

**3. Deploy Actions**
```yaml
# Deploy to GitHub Pages
- uses: peaceiris/actions-gh-pages@v3
  with:
    github_token: ${{ secrets.GITHUB_TOKEN }}
    publish_dir: ./dist

# Deploy to AWS
- uses: aws-actions/configure-aws-credentials@v2
  with:
    aws-access-key-id: ${{ secrets.AWS_ACCESS_KEY_ID }}
    aws-secret-access-key: ${{ secrets.AWS_SECRET_ACCESS_KEY }}
    aws-region: us-east-1
```

**4. Notifications**
```yaml
# Slack notification
- uses: slackapi/slack-github-action@v1
  with:
    webhook-url: ${{ secrets.SLACK_WEBHOOK }}
    payload: |
      {
        "text": "Deployment completed!"
      }
```

---

## 11. Status Badges

Add workflow status to README:

### Badge Syntax

```markdown
![Workflow Name](https://github.com/USERNAME/REPO/workflows/WORKFLOW_NAME/badge.svg)

Or with branch:
![CI](https://github.com/USERNAME/REPO/actions/workflows/ci.yml/badge.svg?branch=main)
```

### Example

In [ ]:
# Add to README.md:
# # My Project
#
# [![CI](https://github.com/user/repo/actions/workflows/ci.yml/badge.svg)](https://github.com/user/repo/actions/workflows/ci.yml)
# [![codecov](https://codecov.io/gh/user/repo/branch/main/graph/badge.svg)](https://codecov.io/gh/user/repo)
# [![License](https://img.shields.io/badge/license-MIT-blue.svg)](LICENSE)
#
# Project description...

---

## 12. Debugging Workflows

### Enable Debug Logging

**Repository secrets:**
```
ACTIONS_RUNNER_DEBUG = true
ACTIONS_STEP_DEBUG = true
```

### Debug Steps

In [ ]:
%%writefile .github/workflows/debug.yml
name: Debug Example

on: [push]

jobs:
  debug:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      
      # Print environment variables
      - name: Print env
        run: env | sort
      
      # Print GitHub context
      - name: Dump GitHub context
        run: echo '${{ toJSON(github) }}'
      
      # Print runner context
      - name: Dump runner context
        run: echo '${{ toJSON(runner) }}'
      
      # List files
      - name: List files
        run: |
          pwd
          ls -la
      
      # Check installed tools
      - name: Check tools
        run: |
          python --version
          node --version
          git --version

### Common Issues

**1. Workflow not triggering**
- Check branch name matches trigger
- Verify YAML syntax
- Check file location: `.github/workflows/`

**2. Permission errors**
```yaml
permissions:
  contents: read
  packages: write
```

**3. Secret not found**
- Verify secret name exactly matches
- Secrets are case-sensitive
- Check secret scope (repo vs environment)

---

## 13. Best Practices

### Do's

1. **Use specific action versions**
   ```yaml
   # Good
   uses: actions/checkout@v3.5.2
   
   # Also acceptable
   uses: actions/checkout@v3
   
   # Avoid
   uses: actions/checkout@main
   ```

2. **Cache dependencies**
   - Dramatically speeds up workflows
   - Use built-in caching when available

3. **Run jobs in parallel when possible**
   - Only use `needs:` when truly dependent
   - Parallel jobs are faster

4. **Use matrix testing**
   - Test multiple versions simultaneously
   - Catch compatibility issues early

5. **Secure secrets properly**
   - Never log secrets
   - Use environment-specific secrets
   - Rotate secrets regularly

### Don'ts

1. **Don't run workflows on every commit**
   ```yaml
   # Avoid for expensive jobs
   on: push
   
   # Better
   on:
     push:
       branches: [ main ]
   ```

2. **Don't ignore failed tests**
   ```yaml
   # Bad
   - run: pytest || true
   
   # Good
   - run: pytest
   ```

3. **Don't use overly broad permissions**
   - Grant minimum necessary permissions
   - Use `permissions:` to limit scope

---

## 14. Practice Exercises

### Exercise 1: Create Basic CI

1. Create a Python project with tests
2. Add a workflow that runs tests on push
3. Test multiple Python versions
4. Add a status badge to README

### Exercise 2: Add Code Quality Checks

1. Add linting (pylint/flake8)
2. Add type checking (mypy)
3. Add code coverage (pytest-cov)
4. Upload coverage to Codecov

### Exercise 3: Deployment Pipeline

1. Create a build job
2. Save build artifacts
3. Create a deploy job that uses artifacts
4. Deploy to GitHub Pages

---

## 15. Quick Reference

### Basic Workflow Template

```yaml
name: CI

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

jobs:
  test:
    runs-on: ubuntu-latest
    
    steps:
    - uses: actions/checkout@v3
    
    - name: Set up Python
      uses: actions/setup-python@v4
      with:
        python-version: '3.10'
        cache: 'pip'
    
    - name: Install dependencies
      run: pip install -r requirements.txt
    
    - name: Run tests
      run: pytest
```

### Common Contexts

```yaml
${{ github.ref }}           # refs/heads/main
${{ github.sha }}           # commit SHA
${{ github.actor }}         # username who triggered
${{ github.repository }}    # owner/repo
${{ runner.os }}            # Linux, Windows, macOS
${{ secrets.SECRET_NAME }}  # Access secrets
```

---

## 16. Next Steps

You now understand:

- ✅ CI/CD fundamentals
- ✅ Creating GitHub Actions workflows
- ✅ Setting up automated testing
- ✅ Using secrets and environment variables
- ✅ Working with artifacts and caching
- ✅ Marketplace actions
- ✅ Workflow debugging

**Next Module**: `07_advanced_github_features.ipynb`

Topics:
- Advanced Git commands (rebase, cherry-pick, stash)
- GitHub Pages deployment
- GitHub API integration
- Security features
- Repository insights
- GitHub CLI advanced usage

---